In [ ]:
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from transformers import BartTokenizer
import jax
import random
from tqdm.notebook import tqdm, trange
from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel

In [ ]:
# make sure we use compatible versions
DALLE_REPO = "flax-community/dalle-mini"
DALLE_COMMIT_ID = "4d34126d0df8bc4a692ae933e3b902a1fa8b6114"
# make sure we use compatible versions
VQGAN_REPO = "flax-community/vqgan_f16_16384"
VQGAN_COMMIT_ID = "90cc46addd2dd8f5be21586a9a23e1b95aa506a9"

In [ ]:
# set up model and processor
clip = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Downloading:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/577M [00:00<?, ?B/s]

INFO:absl:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO:absl:Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/842k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/568 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

In [ ]:

    n_predictions = 10

    # create random keys
    seed = random.randint(0, 2 ** 32 - 1)
    key = jax.random.PRNGKey(seed)
    subkeys = jax.random.split(key, num=n_predictions)


    encoded_images = [
        model.generate(**tokenized_prompt, do_sample=True, num_beams=1, prng_key=subkey)
        for subkey in tqdm(subkeys)
    ]

    encoded_images = [img.sequences[..., 1:] for img in encoded_images]

    decoded_images = [
        vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)
    ]

    clipped_images = [img.squeeze().clip(0.0, 1.0) for img in decoded_images]

str_coco_id="""
524981;	a dog sitting on a boat behind the sail.
509839;	a public bathroom that is empty and not in use.
194120;	a cat sitting on a windowsill next to a painted pumpkin.
197329;	some zebras and a giraffe tall brown grass and trees
432637;	a group of people riding a carousel with trees in the background.
298014;	a large architectural structure near a park
250225;	a display of fruit print dishes in a store
484457;	a large stack of square shaped cakes sitting on a lit table.
541039;	a group of people holding umbrellas standing behind a sign for a umbrella drive.
423313;	a group of motorcycles parked in front of a building."""

for _,line in enumerate(str_coco_id.split("\n")[::-1]):
    print(line)
    id_coco, caption= line.split(";")
    # set a prompt
    prompt = caption.strip()

    # tokenize the prompt
    tokenized_prompt = tokenizer(
        prompt, return_tensors="jax", padding="max_length", truncation=True, max_length=128
    )

    n_predictions = 10

    # create random keys
    seed = random.randint(0, 2 ** 32 - 1)
    key = jax.random.PRNGKey(seed)
    subkeys = jax.random.split(key, num=n_predictions)


    encoded_images = [
        model.generate(**tokenized_prompt, do_sample=True, num_beams=1, prng_key=subkey)
        for subkey in tqdm(subkeys)
    ]

    encoded_images = [img.sequences[..., 1:] for img in encoded_images]

    decoded_images = [
        vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)
    ]

    clipped_images = [img.squeeze().clip(0.0, 1.0) for img in decoded_images]


    # convert to image
    images = [
        Image.fromarray(np.asarray(img * 255, dtype=np.uint8)) for img in clipped_images
    ]
    for i,j in enumerate(images):
      j.save("images_"+id_coco+"_"+str(i)+".jpeg")

    # evaluate scores
    # inputs = processor(text=prompt, images=images, return_tensors="np")
    # logits = clip(**inputs).logits_per_image
    # scores = jax.nn.softmax(logits, axis=0).squeeze()  # normalize and sum all scores to 1

    # # rank images by score
    # print(f"Prompt: {prompt}\n")
    # for idx in scores.argsort()[::-1]:
    #     print(f"Score: {scores[idx]}")
    #     display(images[idx])
    #     print()# set up tokenizer and model

tokenizer = BartTokenizer.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)

model = CustomFlaxBartForConditionalGeneration.from_pretrained(
    DALLE_REPO, revision=DALLE_COMMIT_ID
)

vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)

IndentationError: expected an indented block (<ipython-input-5-6fc6058a9413>, line 35)

In [ ]:
caption = "an elephant in the bathroom"
prompt = caption.strip()
tokenized_prompt = tokenizer(
        prompt, return_tensors="jax", padding="max_length", truncation=True, max_length=128
    )



NameError: name 'tokenizer' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f3692ed-5f27-49a4-899a-82a03e72232c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>